### This is the class implementation of the codes. The function-by-function implemetation is contained in the 'food_drug_predict' file.

In [70]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [71]:
class Predicts:
    '''
    This class harbours all the functions necessary for making predictions in this work.
    
    Params:
    1. dbase - A csv database of object fingerprints with their corresponding name labels.
    
    Note: When using this class, make sure you wrap the 'dbase' value in a quotaion mark as: 'dbase.csv'.
    
    2. finger_print - A list containing only the fingerprints of an object to be identified.
    
    Return:
    1. A list of lists of the predicted substance(s) with their averaged values.
    2. A list of lists of the unlikely substances with their averaged values.
    3. A plot of the predicted substance(s) with their averaged values
    4. A plot of the unlikely substances with their averaged values.
    '''       
    def __init__(self, dbase, finger_print):
        
        # Instantiate the arguments
        self.dbase = dbase
        self.finger_print = finger_print
                
        self.df = pd.read_csv(dbase)
            
    
    def val_ave(self):
        '''
        This function performs element-wise addition and averaging of the numerical values of the datasets, 
        substance-by-substance. At the end it produces a dataset of these values together with their corresponding substances.

        Params: df - Dataframe of fingerprints (numerical values) with their corresponding substance labels, inherited from the 
        parent class. One important condition is that all the substances must be scanned for equal number of times.

        Return: Returns a list of lists of the averaged datasets, with their corresponding substance labels.
        '''
        vals_list = np.zeros(18)
        cnt = 0
        num_scan = len(self.df.values)/len(self.df.Substance.unique()) # Number of scans of eah substance
        vals_lists = []

        for subs in list(self.df.Substance.unique()):
            for vals in self.df.values:
                if vals[-1] == subs: # vals[-1] is the last element in the list which is the substance
                    cnt += 1
                    vals_list = np.add(vals_list, np.array(vals[:-1])) # Element-wise addition. vals[:-1] is a list of only the numerical values

                elif cnt == num_scan:
                    break

            vals_list /= num_scan # Element-wise averaging
            vals_list = np.around(list(vals_list))
            vals_list = list(vals_list)
            vals_list = [math.trunc(fin_list) for fin_list in vals_list] # Remove decimal points
            vals_list.append(subs)
            vals_lists.append(vals_list)
            cnt = 0
            vals_list = np.zeros(18)

        return  vals_lists
        

SyntaxError: 'break' outside loop (Temp/ipykernel_5680/284259945.py, line 28)

In [ ]:
milks3 = [129, 235, 178, 299, 3593, 7257, 434, 516, 15344, 605, 1823, 327, 351, 266, 366, 307, 5096]
#milks3 = 23
vals_lists = Predicts('objdbase.csv', milks3)
print(vals_lists.val_ave())

In [ ]:
a = ['name', 'ok']
type(a)